In [2]:
import pandas as pd
import glob
from datetime import datetime
import matplotlib.pyplot as plt

In [3]:
# helper functions for log parsing

def pass_no_pass(lines):
    for line in lines:
        if "PASSED" in line:
            return True
        elif "FAILED" in line:
            return False
    return

def parse_fp_for_int_diff(lines, mmb_static):
    vals = {'expected': [], 'actual': []}
    delimiters = [" ", "\n"]
    for i in range(len(lines)):
        if "INT" in lines[i]:
            for delimiter in delimiters:
                line = " ".join(lines[i+1].split(delimiter))
            split = list(map(int, line.split()))
            if ("Expected value of" in lines[i]) and ("in" not in lines[i]):
                # print(lines[i])
                vals['expected'].append(split)
            elif "Last value of" in lines[i]:
                # print(lines[i])
                vals['actual'].append(split)

    diffs = []
    for i in range(len(vals['actual'])):
        for j in range(len(vals['actual'][i])):
            if mmb_static:
                if 65535 != vals['actual'][i][j]:
                    diffs.append(int(vals['expected'][i][j]) - int(vals['actual'][i][j]))
                if vals['expected'][i][j] != vals['actual'][i][j]:
                    diffs.append(int(vals['expected'][i][j]) - int(vals['actual'][i][j]))
    return diffs

In [4]:
# create csv of testing logs
# change bool to generate csv if not created
gen_csv = True

def generate_csv():

    df = pd.DataFrame(columns=["kind", "run", "log", "lines", "time", "name", "path", "iters", "expected_vs_actual", "diff_count", "passed"])

    for filename in glob.iglob('**', recursive=True):
        # skip exceptions, keep .txts with logs
        if "log" not in filename or filename[-4:] != ".txt" or "exception" in filename:
            continue
        tmp = filename.split('/')
        if not len(tmp) - 1:
            continue
        fp = open(filename, 'r')
        lines = fp.readlines()
        fp.close()

        kind, name = tmp[0], tmp[-1]

        # TODO: something with GXL logs
        if kind == "gxld":
            continue

        # get iteration count
        iter = [line for line in lines if "Exited" in line][0].split()[-1]
        # get count of i/o matches
        if kind == "mmb_static":
            int_diff = parse_fp_for_int_diff(lines, True)
        else:
            int_diff = parse_fp_for_int_diff(lines, False)
        
        # get pass/fail
        passed = pass_no_pass(lines)


        # parse datetime from file name
        try:
            time = datetime.strptime(filename[-21:-4], '%Y%m%d_%H-%M-%S') # everything but GXLD logs
        except Exception:
            try:
                time = datetime.strptime(filename[-18:-4], '%m-%d-%y_%H-%M') # specifically for GXLD logs
            except Exception as e:
                raise(e)
        
        # get 'run' and 'log' numbers
        runlog = name.split('_')[2]
        run, log = [int(x) for x in runlog.split('run')[-1].split('log')]

        row = {"kind": kind, "run":run, "log": log, "lines": len(lines), "iters": int(iter), "time": time, "name": name,\
            "path": filename, "expected_vs_actual": int_diff, "diff_count": len(int_diff), "passed": passed}
        if df.name.str.contains(name).any():
            continue
        else:
            df.loc[len(df), :] = row
    df = df.sort_values(by=['kind', 'run', 'log']).reset_index(drop=True)
    df.to_csv("./results.csv", index=False)
if gen_csv:
    generate_csv()
df = pd.read_csv("./results.csv")
df = df.sort_values('time')
if "Unnamed: 0" in df.columns:
    df = df.drop("Unnamed: 0", axis=1)

In [5]:
df[df.kind == 'mmb_static']

,kind,run,log,lines,time,name,path,iters,expected_vs_actual,diff_count,passed
701,mmb_static,1,1,15,2023-11-12 23:47:46,log_staticMMBtest_run1log1_20231112_23-47-46.txt,mmb_static/_logs/log_staticMMBtest_run1log1_20...,0,"[17767, 9158, 39017, 18547, 56401, 23807, 3796...",32766,True
702,mmb_static,2,2,30,2023-11-12 23:48:07,log_staticMMBtest_run2log2_20231112_23-48-07.txt,mmb_static/_logs/log_staticMMBtest_run2log2_20...,1,"[-43928, -43928, -52522, -52522, -26503, -2650...",65332,False
706,mmb_static,4,4,30,2023-11-13 00:43:04,log_staticMMBtest_run4log4_20231113_00-43-04.txt,mmb_static/log_staticMMBtest_run4log4_20231113...,1,"[-43928, -43928, -52537, -52537, -26503, -2650...",65320,False
707,mmb_static,4,5,31,2023-11-13 00:46:20,log_staticMMBtest_run4log5_20231113_00-46-20.txt,mmb_static/_logs/log_staticMMBtest_run4log5_20...,2,"[-43928, -43928, -52537, -52537, -26503, -2650...",65322,False
708,mmb_static,4,6,31,2023-11-13 00:50:29,log_staticMMBtest_run4log6_20231113_00-50-29.txt,mmb_static/_logs/log_staticMMBtest_run4log6_20...,2,"[-43928, -43928, -52537, -52537, -26503, -2650...",65322,False
709,mmb_static,7,7,30,2023-11-13 00:53:44,log_staticMMBtest_run7log7_20231113_00-53-44.txt,mmb_static/_logs/log_staticMMBtest_run7log7_20...,2,"[-43928, -43928, -52537, -52537, -26503, -2650...",65324,False
710,mmb_static,7,8,31,2023-11-13 00:56:44,log_staticMMBtest_run7log8_20231113_00-56-44.txt,mmb_static/_logs/log_staticMMBtest_run7log8_20...,2,"[-43928, -43928, -52537, -52537, -26503, -2650...",65324,False
711,mmb_static,9,9,15,2023-11-13 00:56:52,log_staticMMBtest_run9log9_20231113_00-56-52.txt,mmb_static/_logs/log_staticMMBtest_run9log9_20...,0,"[17767, 9158, 39017, 18547, 56401, 23807, 3796...",32766,True
703,mmb_static,2,2,30,2023-11-13 06:12:13,log_staticMMBtest_run2log2_20231113_06-12-13.txt,mmb_static/log_staticMMBtest_run2log2_20231113...,1,"[-43928, -43928, -52537, -52537, -26503, -2650...",65322,False
704,mmb_static,2,3,31,2023-11-13 06:12:38,log_staticMMBtest_run2log3_20231113_06-12-38.txt,mmb_static/log_staticMMBtest_run2log3_20231113...,2,"[-43928, -43928, -52537, -52537, -26503, -2650...",65322,False


In [21]:
tamu_df = pd.read_excel("/home/jacob/GSI_TAMU_202311_scrubbed.xlsx", skiprows=1)
tmp = tamu_df[["Run number", "Run Stop", "Pause #", "Test Mode", "SEU Count (16-bit word errors)"]]
tmp = tmp[tmp["Test Mode"] == "L1 Static"]
y = tmp["Run Stop"]

In [18]:
l1_s = df[df.kind == "l1_static"].sort_values('time')

In [19]:
l1_s

,kind,run,log,lines,time,name,path,iters,expected_vs_actual,diff_count,passed
14,l1_static,1,1,15,2023-11-12 23:41:32,log_staticL1test_run1log1_20231112_23-41-32.txt,l1_static/_logs/log_staticL1test_run1log1_2023...,1,[],0,True
18,l1_static,3,3,64,2023-11-12 23:59:15,log_staticL1test_run3log3_20231112_23-59-15.txt,l1_static/_logs/log_staticL1test_run3log3_2023...,2,"[-2, 16384, -32, -32768, -2048, 4, -32, 32, 12...",62,False
19,l1_static,3,4,65,2023-11-13 00:08:22,log_staticL1test_run3log4_20231113_00-08-22.txt,l1_static/log_staticL1test_run3log4_20231113_0...,3,"[-2, 16384, -32, -32768, -2048, 4, -32, 32, 12...",62,False
20,l1_static,3,5,65,2023-11-13 00:09:08,log_staticL1test_run3log5_20231113_00-09-08.txt,l1_static/_logs/log_staticL1test_run3log5_2023...,3,"[-2, 16384, -32, -32768, -2048, 4, -32, 32, 12...",62,False
22,l1_static,6,6,15,2023-11-13 00:14:17,log_staticL1test_run6log6_20231113_00-14-17.txt,l1_static/_logs/log_staticL1test_run6log6_2023...,1,[],0,True
24,l1_static,6,7,16,2023-11-13 00:14:43,log_staticL1test_run6log7_20231113_00-14-43.txt,l1_static/_logs/log_staticL1test_run6log7_2023...,2,[],0,True
25,l1_static,8,8,64,2023-11-13 00:17:17,log_staticL1test_run8log8_20231113_00-17-17.txt,l1_static/_logs/log_staticL1test_run8log8_2023...,1,"[2048, 4096, -32768, -4096, -4, -16384, -4, -1...",91,False
26,l1_static,8,9,65,2023-11-13 00:17:41,log_staticL1test_run8log9_20231113_00-17-41.txt,l1_static/log_staticL1test_run8log9_20231113_0...,2,"[2048, 4096, -32768, -4096, -4, -16384, -4, -1...",91,False
27,l1_static,8,10,65,2023-11-13 00:19:09,log_staticL1test_run8log10_20231113_00-19-09.txt,l1_static/_logs/log_staticL1test_run8log10_202...,2,"[-12046, 44169, -23452, -278, -35643, 26647, 2...",32768,False
28,l1_static,8,11,65,2023-11-13 00:19:51,log_staticL1test_run8log11_20231113_00-19-51.txt,l1_static/_logs/log_staticL1test_run8log11_202...,3,[],0,False


In [23]:
tamu = tamu_df.copy()
tamu.columns

Index(['Run number', 'Run start date and time', 'Run Stop',
       'Duration of the run', 'Selected beam', 'Effective LET (MeVcm2/mg)',
       'Effective fluence (ions/cm2)', 'Average flux (ions/(cm2s))',
       'PCB - DUT', 'DUT Thickness (um)', 'Test Mode', 'Pre-Run Notes',
       'Main Rail (12V) I (A)', 'Pause #', 'Pause Notes', 'Pause Valid?',
       'Run Accumulated Fluence\n(ions/cm2)', 'Flunece Check',
       'Pause Fluence (ions/cm2)', 'Requested Flus',
       'Highest Observed 12V Rail Current I (A)', 'SEFI Type',
       'Mitigation Applied end of Pause', 'Run Notes/ Mitigation Steps',
       'Valid for DSEE', 'DUT\nDSEE Observed?', 'Valid for SEU',
       'SEU Count (16-bit word errors)', 'Stuck bits', 'Valid for SEE',
       'SEFI Description', 'Device SEFI'],
      dtype='object')

In [24]:
tamu['Test Mode'].value_counts()

Test Mode
GXLB (KNN)          41
L1 Static           11
L1 Dynamic Read      9
MMB Static           8
MMB Dynamic Read     7
Idle SEL             3
Name: count, dtype: int64

In [25]:
df.kind.value_counts()

kind
mmb_dynamic    668
l1_static       18
l1_dynamic      14
mmb_static      11
l1s_temp         1
Name: count, dtype: int64

In [27]:
df[df.kind == 'mmb_static']

,kind,run,log,lines,time,name,path,iters,expected_vs_actual,diff_count,passed
701,mmb_static,1,1,15,2023-11-12 23:47:46,log_staticMMBtest_run1log1_20231112_23-47-46.txt,mmb_static/_logs/log_staticMMBtest_run1log1_20...,0,"[17767, 9158, 39017, 18547, 56401, 23807, 3796...",32766,True
702,mmb_static,2,2,30,2023-11-12 23:48:07,log_staticMMBtest_run2log2_20231112_23-48-07.txt,mmb_static/_logs/log_staticMMBtest_run2log2_20...,1,"[-43928, -52522, -26503, 14452, -8894, 19952, ...",32768,False
706,mmb_static,4,4,30,2023-11-13 00:43:04,log_staticMMBtest_run4log4_20231113_00-43-04.txt,mmb_static/log_staticMMBtest_run4log4_20231113...,1,"[-43928, -52537, -26503, 14452, -8894, 19952, ...",32768,False
707,mmb_static,4,5,31,2023-11-13 00:46:20,log_staticMMBtest_run4log5_20231113_00-46-20.txt,mmb_static/_logs/log_staticMMBtest_run4log5_20...,2,"[-43928, -52537, -26503, 14452, -8894, 19952, ...",32768,False
708,mmb_static,4,6,31,2023-11-13 00:50:29,log_staticMMBtest_run4log6_20231113_00-50-29.txt,mmb_static/_logs/log_staticMMBtest_run4log6_20...,2,"[-43928, -52537, -26503, 14452, -8894, 19952, ...",32768,False
709,mmb_static,7,7,30,2023-11-13 00:53:44,log_staticMMBtest_run7log7_20231113_00-53-44.txt,mmb_static/_logs/log_staticMMBtest_run7log7_20...,2,"[-43928, -52537, -26503, 14452, -8894, 19952, ...",32768,False
710,mmb_static,7,8,31,2023-11-13 00:56:44,log_staticMMBtest_run7log8_20231113_00-56-44.txt,mmb_static/_logs/log_staticMMBtest_run7log8_20...,2,"[-43928, -52537, -26503, 14452, -8894, 19952, ...",32768,False
711,mmb_static,9,9,15,2023-11-13 00:56:52,log_staticMMBtest_run9log9_20231113_00-56-52.txt,mmb_static/_logs/log_staticMMBtest_run9log9_20...,0,"[17767, 9158, 39017, 18547, 56401, 23807, 3796...",32766,True
703,mmb_static,2,2,30,2023-11-13 06:12:13,log_staticMMBtest_run2log2_20231113_06-12-13.txt,mmb_static/log_staticMMBtest_run2log2_20231113...,1,"[-43928, -52537, -26503, 14452, -8894, 19952, ...",32768,False
704,mmb_static,2,3,31,2023-11-13 06:12:38,log_staticMMBtest_run2log3_20231113_06-12-38.txt,mmb_static/log_staticMMBtest_run2log3_20231113...,2,"[-43928, -52537, -26503, 14452, -8894, 19952, ...",32768,False


In [7]:
masks = []
for path in df[df.kind == "mmb_dynamic"].path:
    lines = open(path).read().split('\n')
    mask, mask_bool = [], False
    for line in lines:
        if "mask" in line:
            mask_bool = True
            continue
        elif ("SECTION" in line) or (line == ""):
            mask_bool = False
        if mask_bool:
            mask.append(line)
    if len(mask): masks.append(mask)

In [114]:
f = open('/home/jacob/radTestLogBackup/mmb_static/log_staticMMBtest_run2log4_20231113_06-13-00.txt')
lines = f.read().split("\n")
f.close()

In [121]:
expected, actual = [], []
delimiters = [" ", "\n"]
for i in range(len(lines)):
    if "INT" in lines[i]:
        for delimiter in delimiters:
            line = " ".join(lines[i+1].split(delimiter))
        split = list(map(int, line.split()))
        split = list(map(lambda x: '{0:016b}'.format(x), split))
        if ("Expected value of" in lines[i]) and ("in" not in lines[i]):
            # print(lines[i])
            expected.append(split)
        elif "Last value of" in lines[i]:
            # print(lines[i])
            actual.append(split)
if not len(expected)-1:
    expected, actual = expected[0], actual[0]

In [19]:
! rm test_bits.txt
f = open("test_bits.txt", "a")
for i in range(len(masks[632])):
    f.write(masks[632][i])
    if i < len(masks[632]) - 1:
        f.write('\n')
f.close()

In [16]:
len(masks[632]), len(masks[632][0])

(48, 32768)

In [123]:
len(expected), len(expected[0])

(32768, 16)

In [205]:
_32k * 16

524288

In [207]:
ncols * nbanks * nsections

131072